In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
latent_dim = 150
image_dim = 28 * 28  # MNIST images are 28x28
batch_size = 60
learning_rate = 0.0002
num_epochs = 50

# Generator
class Generator(nn.Module):
    def __init__(self, latent_dim, image_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, image_dim),
            nn.Tanh()  # Output values in range [-1, 1]
        )

    def forward(self, z):
        return self.model(z)

# Discriminator
class Discriminator(nn.Module):
    def __init__(self, image_dim):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(image_dim, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, 1),
            nn.Sigmoid()  # Output a probability
        )

    def forward(self, x):
        return self.model(x)

# Initialize models
generator = Generator(latent_dim, image_dim).to(device)
discriminator = Discriminator(image_dim).to(device)

# Loss and optimizers
criterion = nn.BCELoss()
g_optimizer = optim.Adam(generator.parameters(), lr=learning_rate)
d_optimizer = optim.Adam(discriminator.parameters(), lr=learning_rate)

# Load MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])  # Scale images to range [-1, 1]
])
dataset = datasets.MNIST(root="./data", train=True, transform=transform, download=True)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Training loop
for epoch in range(num_epochs):
    for batch_idx, (real_images, _) in enumerate(data_loader):
        # Flatten images and move to device
        real_images = real_images.view(-1, image_dim).to(device)
        batch_size = real_images.size(0)

        # Labels for real and fake data
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

        # Train Discriminator
        z = torch.randn(batch_size, latent_dim).to(device)
        fake_images = generator(z)
        
        real_loss = criterion(discriminator(real_images), real_labels)
        fake_loss = criterion(discriminator(fake_images.detach()), fake_labels)
        d_loss = real_loss + fake_loss

        d_optimizer.zero_grad()
        d_loss.backward()
        d_optimizer.step()

        # Train Generator
        z = torch.randn(batch_size, latent_dim).to(device)
        fake_images = generator(z)
        g_loss = criterion(discriminator(fake_images), real_labels)  # Fool the discriminator

        g_optimizer.zero_grad()
        g_loss.backward()
        g_optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}]  D Loss: {d_loss.item():.4f}  G Loss: {g_loss.item():.4f}")

# Save the trained generator model
torch.save(generator.state_dict(), "generator_2.pth")
print("Generator model saved!")


Epoch [1/20]  D Loss: 0.1504  G Loss: 3.9082
Epoch [2/20]  D Loss: 0.6844  G Loss: 2.3826
Epoch [3/20]  D Loss: 1.1067  G Loss: 2.6726
Epoch [4/20]  D Loss: 0.6959  G Loss: 4.4931
Epoch [5/20]  D Loss: 0.2052  G Loss: 3.4598
Epoch [6/20]  D Loss: 0.1995  G Loss: 2.9064
Epoch [7/20]  D Loss: 0.2921  G Loss: 3.9876
Epoch [8/20]  D Loss: 0.2943  G Loss: 7.8975
Epoch [9/20]  D Loss: 0.2136  G Loss: 4.7095
Epoch [10/20]  D Loss: 0.1457  G Loss: 6.3043
Epoch [11/20]  D Loss: 0.4137  G Loss: 5.1909
Epoch [12/20]  D Loss: 0.4034  G Loss: 4.6716
Epoch [13/20]  D Loss: 0.1770  G Loss: 4.1958
Epoch [14/20]  D Loss: 0.2680  G Loss: 3.8860
Epoch [15/20]  D Loss: 0.3845  G Loss: 4.3139
Epoch [16/20]  D Loss: 0.1789  G Loss: 4.7628
Epoch [17/20]  D Loss: 0.2977  G Loss: 3.4236
Epoch [18/20]  D Loss: 0.3295  G Loss: 4.3838
Epoch [19/20]  D Loss: 0.4413  G Loss: 4.0398
Epoch [20/20]  D Loss: 0.2043  G Loss: 3.7852
Generator model saved!


In [6]:
import matplotlib.pyplot as plt

# Load the trained generator model (if not already loaded)
generator.load_state_dict(torch.load("generator_2.pth"))
generator.eval()

# Generate random noise and create fake images
z = torch.randn(16, latent_dim).to(device)  # Generate 16 samples
fake_images = generator(z).detach().cpu()  # Detach from computation graph

# Reshape and scale images back to [0, 1]
fake_images = fake_images.view(-1, 28, 28)
fake_images = (fake_images + 1) / 2.0

# Plot the generated images
fig, axes = plt.subplots(4, 4, figsize=(8, 8))
for i, ax in enumerate(axes.flatten()):
    ax.imshow(fake_images[i], cmap="gray")
    ax.axis("off")
plt.show()


NameError: name 'generator' is not defined